In [46]:
import networkx as nx
import numpy as np
import copy


def opinion_diff(u, v, G):
    return G.nodes[u]['opinion'] - G.nodes[v]['opinion']


def update_opinions(u, v, G, convergence, symmetric_updating=True):
    diff = opinion_diff(u, v, G)

    G.nodes[u]['opinion'] -= convergence * diff
    
    if symmetric_updating:
        G.nodes[v]['opinion'] += convergence * diff

        
def update_effective_net(u, v, effective_net, G, threshold, symmetric_updating):
    # update u's neighbors
    for w in G.neighbors(u):
        diff = abs(opinion_diff(u, w, G))
        
        if diff <= threshold and not effective_net.has_edge(u, w):
            effective_net.add_edge(u, w)

        elif diff > threshold and effective_net.has_edge(u, w):
            effective_net.remove_edge(u, w)
        
    if not symmetric_updating:
        return
    
    # update v's neighbors
    for w in G.neighbors(v):
        diff = abs(opinion_diff(u, w, G))
        
        if diff <= threshold and not effective_net.has_edge(v, w):
            effective_net.add_edge(v, w)

        elif diff > threshold and effective_net.has_edge(v, w):
            effective_net.remove_edge(v, w)


def simulate_deffuant_model(G, n_iter=100, threshold=0.5, convergence=0.5, symmetric_updating=True):
    """
    Simulates the Deffuant model on the given graph.

    Parameters
    ----------
    G : networkx.Graph
        The graph on which to simulate the model.
    n_iter : int
        The number of iterations to run the simulation for.
    threshold : float
        The threshold value used in the model.
    convergence : float
        The convergence value used in the model.

    Returns
    -------
    list of networkx.Graph
        A list of graphs, where each graph represents the state of the network at a given time step.
    """
    
    # initialize node attributes
    for node in G.nodes():
        G.nodes[node]['opinion'] = np.random.uniform(0, 1)

    # get initial effective network
    effective_net = copy.deepcopy(G)
    
    # remove edges between nodes with opinion difference above threshold
    for u, v in effective_net.edges:
        if abs(opinion_diff(u, v, G)) > threshold:
            effective_net.remove_edge(u, v)

    # initialize the list of nets that will be returned
    effective_networks = [effective_net]

    # run simulation
    for t in range(1, n_iter):
        # copy last effective network
        effective_net = copy.deepcopy(effective_networks[t-1])
        
        # choose random node to update
        u = np.random.choice(G.order())
        
        # get another node if u is isolated
        while not list(effective_net.neighbors(u)):
            u = np.random.choice(G.order())
        
        # get a random neighbor of u
        v = np.random.choice(list(effective_net.neighbors(u)))

        # update their opinions
        update_opinions(u, v, G, convergence, symmetric_updating)
        
        # we now update the connections of these nodes in the effective network depending on
        # the opinions of their neighbors in the underlying network G
        update_effective_net(u, v, effective_net, G, threshold, symmetric_updating)
        
        # add it to list
        effective_networks.append(effective_net)
            
    return effective_networks

In [47]:
# we set n=900 in the BA graph so that it has the same amount of vertices as the 30x30 grid graph
# we set m=2 so that the average degree of the graph is 2*|E|/n = 2*(3+2*897)/900 ~= 4 as in the grid graph

BA_graph = nx.barabasi_albert_graph(n=900, m=2, seed=42, initial_graph=nx.complete_graph(3))
lattice  = nx.grid_2d_graph(30, 30, periodic=False, create_using=None)

In [48]:
res = simulate_deffuant_model(BA_graph, 1000, 0.3, 0.3)

In [44]:
nx.draw(res[-1], nx.spring_layout(res[-1]))

AttributeError: module 'scipy.sparse' has no attribute 'coo_array'

In [4]:
print(nx.__version__)

2.8.8


In [45]:
G = nx.Graph()
G.add_edge(1,2)

G.nodes[1]['a'] = 42
print(G.nodes[1]['a'])

def change(n, key, val, G):
    G.nodes[n][key] = val
    
change(1,'a', 43, G)

print(G.nodes[1]['a'])

42
43
